# Phishing Websites Classification

According to `cybercrime.org` phishing is a type of cybercrime that involves the use of fake e-mail addresses and websites that trick people into giving away their personal information. The Phishing Websites data set is a dataset that contains data about different websites that either are or are not malicious in stealing someone's data. In this project, I will be predicting if a certain URL is proven to be malicious or not using `XGBoost`.

Let's get to work!

In [1]:
# Importing Dependencies and Fetching the Dataset
from ucimlrepo import fetch_ucirepo
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# fetch dataset 
phisihing_data = fetch_ucirepo(id=327) 
  
# data (as pandas dataframes) 
df = pd.DataFrame(data = phisihing_data.data['features'], columns = phisihing_data.feature_names)
df['target'] = phisihing_data.data['targets']

# Extract features and target
X = df.drop(columns=['target'])
y = df['target'] = df['target'].map({-1: 0, 1: 1})

df.head()

,having_ip_address,url_length,shortining_service,having_at_symbol,double_slash_redirecting,prefix_suffix,having_sub_domain,sslfinal_state,domain_registration_length,favicon,...,popupwindow,iframe,age_of_domain,dnsrecord,web_traffic,page_rank,google_index,links_pointing_to_page,statistical_report,target
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,0
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,0
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,0
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,0
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1


In [2]:
# Going to Do Some EDA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11055 entries, 0 to 11054
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   having_ip_address           11055 non-null  int64
 1   url_length                  11055 non-null  int64
 2   shortining_service          11055 non-null  int64
 3   having_at_symbol            11055 non-null  int64
 4   double_slash_redirecting    11055 non-null  int64
 5   prefix_suffix               11055 non-null  int64
 6   having_sub_domain           11055 non-null  int64
 7   sslfinal_state              11055 non-null  int64
 8   domain_registration_length  11055 non-null  int64
 9   favicon                     11055 non-null  int64
 10  port                        11055 non-null  int64
 11  https_token                 11055 non-null  int64
 12  request_url                 11055 non-null  int64
 13  url_of_anchor               11055 non-null  int64
 14  links_

In [3]:
# Checking for missing values
df.isnull().sum()

having_ip_address             0
url_length                    0
shortining_service            0
having_at_symbol              0
double_slash_redirecting      0
prefix_suffix                 0
having_sub_domain             0
sslfinal_state                0
domain_registration_length    0
favicon                       0
port                          0
https_token                   0
request_url                   0
url_of_anchor                 0
links_in_tags                 0
sfh                           0
submitting_to_email           0
abnormal_url                  0
redirect                      0
on_mouseover                  0
rightclick                    0
popupwindow                   0
iframe                        0
age_of_domain                 0
dnsrecord                     0
web_traffic                   0
page_rank                     0
google_index                  0
links_pointing_to_page        0
statistical_report            0
target                        0
dtype: i

In [4]:
# Checking for Class Imbalance
df['target'].value_counts()

target
1    6157
0    4898
Name: count, dtype: int64

In [5]:
df.shape

(11055, 31)

In [17]:
# Making the pipeline to scale and encode the data
X_df = df.drop(columns=['target'])
numeric_features = X_df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_df.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
X = preprocessor.fit_transform(X_df)

# Splitting the data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2, random_state=42)

In [18]:
X_train.shape, X_test.shape

((8844, 30), (2211, 30))

In [19]:
y_train.shape, y_test.shape

((8844,), (2211,))

In [20]:
y_test

6461     0
10463    0
3390     1
4424     0
6256     0
        ..
8048     1
1959     0
9450     1
3252     1
513      1
Name: target, Length: 2211, dtype: int64

In [21]:
# Converting the data into DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [22]:
# Making the model and defining the parameters

params = {
    'max_depth': 3,
    'objective': 'multi:softmax',
    'num_class': 3,
    'n_gpus': 1,
    'device': 'cuda',
    'tree_method': 'hist',
    'learning_rate': 0.1,
    'random_state': 42
}

n = 100

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

c:\Users\neche\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:00:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_gpus" } are not used.

  warnings.warn(smsg, UserWarning)


In [23]:
# Making Predictions with the model
y_pred = model.predict(dtest)

In [24]:
# Using Classification Report to evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 919   61]
 [  47 1184]]
              precision    recall  f1-score   support

           0       0.95      0.94      0.94       980
           1       0.95      0.96      0.96      1231

    accuracy                           0.95      2211
   macro avg       0.95      0.95      0.95      2211
weighted avg       0.95      0.95      0.95      2211

